In [1]:
import pandas as pd
import statsmodels.formula.api as smf

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_16608\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Вернёмся к задаче Cookie cats из первой лекции и к большой дисперсии её данных.

In [26]:
data = pd.read_csv("../data/cookie_cats.csv")
data

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [27]:
data.groupby('version').sum_gamerounds.agg(['var', 'mean'])

,var,mean
version,,
gate_30,65903.321897,52.456264
gate_40,10669.736422,51.298776


Предварительно удалите один максимум.

In [28]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
data = data[data.sum_gamerounds < data.sum_gamerounds.max()]
data.groupby('version').sum_gamerounds.agg(['var', 'mean'])

,var,mean
version,,
gate_30,10415.753288,51.342111
gate_40,10669.736422,51.298776


Посмотрим, насколько можно сократить дисперсию с помощью стратификации и CUPED

### Стратификация

Для деления на 4 страты используйте retention (да, потому что в датасете больше ничего нет)<br>
Найдите и сохраните дисперсию и среднее для групп `gate_30` и `gate_40`

In [32]:
A = data[data.version == 'gate_30']
B = data[data.version == 'gate_40']

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

A_strats = A.groupby(['retention_1', 'retention_7']).sum_gamerounds.agg(['var', 'mean', ('weight', lambda x: len(x) / len(A))])
B_strats = B.groupby(['retention_1', 'retention_7']).sum_gamerounds.agg(['var', 'mean', ('weight', lambda x: len(x) / len(B))])
A_strat_var = (A_strats['var'] * A_strats['weight']).sum()
B_strat_var = (B_strats['var'] * B_strats['weight']).sum()
A_strat_mean = (A_strats['mean'] * A_strats['weight']).sum()
B_strat_mean = (B_strats['mean'] * B_strats['weight']).sum()

A_strat_var, B_strat_var

(6974.336965830477, 7080.3682947359775)

### CUPAC

Исторические данные для CUPED есть не всегда, но в качестве X можно использовать что угодно, сокращающее $1 - cor^2(X, Y)$<br>
...